**LEITURA DOS DADOS DO SUSTAIN BENCH**

In [46]:
import numpy as np
import pandas as pd

# -----------------------------
# Função para carregar um .npz
# -----------------------------
def load_npz_array(path):
    data = np.load(path, allow_pickle=True) # Adiciona allow_pickle=True
    return data[data.files[0]]  # pega o array interno


# -----------------------------
# 1. Carregar todos os arquivos de treino
# -----------------------------
X_hist = load_npz_array("train_hists.npz")
X_ndvi = load_npz_array("train_ndvi.npz")
X_loc = load_npz_array("train_locs.npz")
X_year = load_npz_array("train_years.npz")
X_key = load_npz_array("train_keys.npz")
y = load_npz_array("train_yields.npz")

# -----------------------------
# 2. Mostrar shapes
# -----------------------------
print("Shapes dos arrays:")
print("Hist:", X_hist.shape)
print("NDVI:", X_ndvi.shape)
print("Locs:", X_loc.shape)
print("Years:", X_year.shape)
print("Keys:", X_key.shape)
print("Yields:", y.shape)


# -----------------------------
# 3. Montar um DataFrame para visualização
# -----------------------------

# Hist tem muitas colunas → renomear
X_hist_reshaped = X_hist.reshape(X_hist.shape[0], -1) # Achata as dimensões restantes
hist_cols = [f"hist_{i}" for i in range(X_hist_reshaped.shape[1])]
df_hist = pd.DataFrame(X_hist_reshaped, columns=hist_cols)

# NDVI também
ndvi_cols = [f"ndvi_{i}" for i in range(X_ndvi.shape[1])]
df_ndvi = pd.DataFrame(X_ndvi, columns=ndvi_cols)


# Para corrigir o ValueError, vamos criar um DataFrame com duas colunas de None.
df_loc = pd.DataFrame({'lat': X_loc, 'lon': X_loc})

df_year = pd.DataFrame(X_year, columns=["year"])
df_key = pd.DataFrame(X_key, columns=["region_id"])
df_y = pd.DataFrame(y, columns=["yield"])

# Junta tudo
df = pd.concat([df_hist, df_ndvi, df_loc, df_year, df_key, df_y], axis=1)

print("\nPrimeiras linhas:")
print(df.head())

print("\nColunas totais:")
print(df.columns)

#printa so as locs


Shapes dos arrays:
Hist: (247, 32, 32, 9)
NDVI: (247, 32)
Locs: (247,)
Years: (247,)
Keys: (247,)
Yields: (247,)

Primeiras linhas:
     hist_0    hist_1    hist_2   hist_3    hist_4    hist_5    hist_6  \
0  0.000000  0.000270  0.000025  0.00000  0.000037  0.000061  0.000061   
1  0.000000  0.000945  0.000000  0.00000  0.000065  0.000126  0.000189   
2  0.000673  0.000657  0.000000  0.00003  0.000547  0.000437  0.002556   
3  0.000000  0.000538  0.000107  0.00000  0.000401  0.000000  0.000000   
4  0.000000  0.000343  0.000000  0.00000  0.000144  0.000274  0.000480   

   hist_7  hist_8    hist_9  ...   ndvi_27   ndvi_28   ndvi_29   ndvi_30  \
0     0.0     0.0  0.000000  ...  0.698605  0.640461  0.566693  0.645517   
1     0.0     0.0  0.000000  ...  0.822922  0.786009  0.720077  0.574686   
2     0.0     0.0  0.000485  ...  0.637951  0.673795  0.624429  0.531740   
3     0.0     0.0  0.000000  ...  0.729741  0.787198  0.824194  0.805108   
4     0.0     0.0  0.000000  ...  0.699269 

# **ACHAR INFORMAÇOES ESPACIAIS**

O conjunto de dados carece de Latitude e Longitude, o que é um impedimento crucial.

A Solução é: Derivar o Lat/Long do REGION_ID usando o nome do município e a ferramenta Geopy.

A Justificativa Principal é: Embora as features existentes já sejam climáticas históricas, a adição do Lat/Long não só oferece um contexto geográfico direto, mas, mais importante, serve como chave de agregação para incorporar dados climáticos externos mais ricos e detalhados (ex: interpolação de dados do INMET), maximizando assim o poder preditivo do modelo.

In [7]:
# Instala a biblioteca geopy para geocodificação
!pip install geopy

In [8]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time

geolocator = Nominatim(user_agent="my-geocoder-app")

def get_lat_lon(region_id):
    try:
        # Extrai o nome da região (removendo o ano e '_brasil_')
        region_name = region_id.split('_brasil_')[0].replace('_', ' ').strip()
        location = geolocator.geocode(region_name + ", Brasil", timeout=5) # Adiciona "Brasil" para melhorar a precisão
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except (GeocoderTimedOut, GeocoderServiceError):
        # Lida com erros de timeout ou serviço
        time.sleep(1) # Espera um pouco antes de tentar novamente
        return get_lat_lon(region_id) # Tenta novamente
    except Exception as e:
        print(f"Erro ao geocodificar {region_id}: {e}")
        return None, None

# Aplica a função para obter as coordenadas
df['new_lat'], df['new_lon'] = zip(*df['region_id'].apply(get_lat_lon))

print("Valores únicos de new_lat e new_lon (amostra):")
print(df[['new_lat', 'new_lon']].head())
print(f"\nNúmero de NAs em new_lat: {df['new_lat'].isnull().sum()}")

Valores únicos de new_lat e new_lon (amostra):
     new_lat    new_lon
0        NaN        NaN
1 -22.662089 -50.420623
2 -17.986969 -49.376600
3        NaN        NaN
4        NaN        NaN

Número de NAs em new_lat: 166


In [9]:
# Atualiza as colunas 'lat' e 'lon' com as novas coordenadas
df['lat'] = df['new_lat']
df['lon'] = df['new_lon']

# Remove as colunas temporárias 'new_lat' e 'new_lon'
df = df.drop(columns=['new_lat', 'new_lon'])

print("Colunas 'lat' e 'lon' atualizadas. Primeiras linhas:")
print(df[['lat', 'lon']].head())
print(f"\nNúmero de NAs restantes em lat: {df['lat'].isnull().sum()}")
print(f"Número de NAs restantes em lon: {df['lon'].isnull().sum()}")

Colunas 'lat' e 'lon' atualizadas. Primeiras linhas:
         lat        lon
0        NaN        NaN
1 -22.662089 -50.420623
2 -17.986969 -49.376600
3        NaN        NaN
4        NaN        NaN

Número de NAs restantes em lat: 166
Número de NAs restantes em lon: 166


# Implementaçao do Modelo

In [22]:
!pip install scikit-learn
!pip install streamlit pyngrok

In [31]:
from pyngrok import ngrok

# Coloque seu token aqui
ngrok.set_auth_token("36RRGdxtdsr6l1S2mMDLnCrDNf2_2pxhtaKfNrdsZJAUEM8oq")

In [32]:
%%writefile app.py
import streamlit as st

st.title("Teste Streamlit no Colab")
st.write("Se você está vendo isso, funcionou!")


Writing app.py


In [33]:
df.to_pickle("dados.pkl")


In [47]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# --------------------------
# Carregar DataFrame salvo
# --------------------------
st.title("Modelos de Predição de Produtividade de Soja")

@st.cache_data
def load_data():
    return pd.read_pickle("dados.pkl")

df = load_data()
st.write("Dimensões do dataset:", df.shape)
st.write(df.head())

# --------------------------
# 1. Seleção de alvo e split
# --------------------------

#corta os features com hist da seleçao
available_targets = ["yield"]

# O 'yield' será sempre o target padrão, então o índice é 0
default_target_index = 0

target_col = st.selectbox("Selecione o target", available_targets, index=default_target_index)

X = df.drop(columns=[target_col,'region_id'])
y = df[target_col]

test_size = st.slider("Tamanho do conjunto de teste (%)", 10, 50, 20) / 100
random_state = st.number_input("Random State", value=42)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state
)

# --------------------------
# 2. Seleção do algoritmo
# --------------------------
st.subheader("Escolha o Algoritmo")

model_name = st.selectbox(
    "Modelo",
    [
        "Linear Regression",
        "Random Forest",
        "Gradient Boosting"
    ]
)

# Hiperparâmetros
if model_name == "Random Forest":
    n_estimators = st.slider("Número de árvores", 50, 500, 200)
    max_depth = st.slider("Profundidade máxima", 2, 30, 10)
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

elif model_name == "Gradient Boosting":
    learning_rate = st.slider("Learning Rate", 0.01, 0.5, 0.1)
    n_estimators = st.slider("Número de estimadores", 50, 500, 150)
    model = GradientBoostingRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators
    )

else:
    model = LinearRegression()

# --------------------------
# 3. Treinar modelo
# --------------------------
if st.button("Treinar Modelo"):
    with st.spinner("Treinando..."):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # --------------------------
        # 4. Mostrar métricas
        # --------------------------
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        st.success("Treinamento Concluído!")

        st.subheader("Métricas")
        st.metric("MAE", f"{mae:.3f}")
        st.metric("RMSE", f"{rmse:.3f}")
        st.metric("R²", f"{r2:.3f}")

        # --------------------------
        # 5. Plot Real vs Predito
        # --------------------------
        st.subheader("Real vs Predito")

        df_plot = pd.DataFrame({
            "Real": y_test.values,
            "Predito": y_pred
        })

        st.line_chart(df_plot)


Overwriting app.py


In [48]:
public_url = ngrok.connect(addr="8501")
print("Acesse aqui:", public_url)

!streamlit run soybean_app.py --server.port 8501

Acesse aqui: NgrokTunnel: "https://presecular-yael-unavowably.ngrok-free.dev" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.145.219:8501



  Stopping...
  Stopping...
